In [ ]:
!pip install tld

In [ ]:
import pandas as pd
import numpy as np

from urllib.parse import urlparse
from tld import get_tld
import os.path

import re
from bs4 import BeautifulSoup
import urllib
import urllib.request
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split

## Load Data from different Sources

In [ ]:
df_train = pd.read_csv("../input/dataset-of-malicious-and-benign-webpages/Webpages_Classification_train_data.csv/Webpages_Classification_train_data.csv")
df_test = pd.read_csv("../input/dataset-of-malicious-and-benign-webpages/Webpages_Classification_test_data.csv/Webpages_Classification_test_data.csv")
#Ensuring correct sequence of columns 
df_train = df_train[['url','label']]
df_test = df_test[['url','label']]

df_test['label'].replace(to_replace ="good", value ="benign", inplace=True)
df_train['label'].replace(to_replace ="good", value ="benign", inplace=True)
df_test['label'].replace(to_replace ="bad", value ="malicious", inplace=True)
df_train['label'].replace(to_replace ="bad", value ="malicious", inplace=True)

malicious_dftrain = df_train[df_train['label'] == 'malicious']
malicious_dftest = df_test[df_test['label'] == 'malicious']

In [ ]:
df_test.duplicated().sum()
df_test = df_test.drop_duplicates()

In [ ]:
malicious = pd.concat([malicious_dftrain, malicious_dftest], axis=0)
malicious.to_csv(r'./Malicious1.csv', index = False)
df_35k = pd.read_csv('Malicious1.csv')
df_35k.head()

In [ ]:
df_651K = pd.read_csv('../input/dataset/malicious_phish.csv')
df_651K.rename(columns = {'type':'label'}, inplace = True)

df_651K.head()

In [ ]:
df_450K = pd.read_csv('../input/dataset/urldata.csv')
df_450K.drop(['Column1', 'result'], axis=1, inplace=True)

df_450K.head()

In [ ]:
df_76K = pd.read_csv('../input/dataset/Spam_data.csv')
df_76K.head()

In [ ]:
df_8K = pd.read_csv('../input/dataset/PhishTank.csv')
df_8K.head()

In [ ]:
df_500 = pd.read_csv('../input/dataset/OpenPhish.csv')
df_500.head()

In [ ]:
df_56K =  pd.read_csv('../input/dataset/collected_data.csv')
print(df_56K.info())
df_56K_MAL = df_56K[df_56K['class'] == 1]
df_56K_MAL.head()

In [ ]:
df_549K =  pd.read_csv('../input/dataset/phishing_site_urls.csv')
# print(df_549K.info())
df_549K_MAL = df_549K[df_549K['label'] == 'bad']
df_549K_MAL['label'].replace(to_replace ="bad", value ="phishing", inplace=True)
df_56K_MAL = df_56K_MAL[['url', 'label']]
df_549K_MAL.head()

In [ ]:
All_data = pd.concat([df_651K, df_450K, df_76K, df_8K, df_500, df_56K_MAL, df_35k, df_549K_MAL]).drop_duplicates('url').reset_index(drop=True)
All_data

In [ ]:
All_data.groupby('label').count()

## Data Cleansing

In [ ]:
#Checking Missing Values
All_data.isnull().sum()

In [ ]:
total = 0
def clean_functions(url):
    global total
    re1 = re.compile(r"[]") 
#     re1 = re.compile("^[A-Za-z0-9|!|@|#|$|%|^|&|*|(|)|-|+|~]$")  
    if re1.search(url):
        All_data.drop(All_data.loc[All_data['url'] == url].index, inplace=True)
        print(url)
        total += 1
    
    
All_data['url'].apply(lambda i: clean_functions(i))
print(total)

In [ ]:
All_data['url'] = All_data['url'].replace(" ", "", regex=True)

In [ ]:
count = All_data.label.value_counts()

sns.barplot(x=count.index, y=count)
plt.xlabel('Types')
plt.ylabel('Count');

In [ ]:
a = {'benign' : 0, 'defacement' : 1, 'malicious': 1, 'malware': 1, 'phishing': 1, 'spam': 1}
All_data['Class'] = All_data['label'].map(a)

All_data.head()

##  Feature Extraction 

**1- Lexical Features**

**Length Features**

In [ ]:
urldata = All_data.copy()

In [ ]:
#Length of URL
urldata['url_length'] = urldata['url'].apply(lambda i: len(str(i)))

In [ ]:
#Hostname Length

# urldata['hostname_length'] = urldata['url'].apply(lambda i: len(urlparse(i).netloc))

def process_tld(url):
    try:
        res = get_tld(url, as_object = True, fail_silently=False,fix_protocol=True)
        pri_domain= res.parsed_url.netloc
        domain_len = len(pri_domain)
    except :
        pri_domain= None
    return pri_domain

urldata['hostname'] = urldata['url'].apply(lambda i: process_tld(i))

def host_len(i):
    try:
        domain_len = len(i)
    except :
        domain_len = 0
    return domain_len
urldata['hostname_length'] = urldata['hostname'].apply(lambda i: host_len(i))

In [ ]:
#Path Length

# urldata['path_length'] = urldata['url'].apply(lambda i: len(urlparse(i).path))

def get_path(url):
    try:
        res = get_tld(url, as_object = True, fail_silently=False, fix_protocol=True)
        length = len(res.parsed_url.path)
    except:
        length = 0
    return length

urldata['path_length'] = urldata['url'].apply(lambda i: get_path(i))

In [ ]:
#Length of Top Level Domain
def get_tld_Name(url):
    try:
        k = get_tld(url, fail_silently=True, fix_protocol=True)
    except:
        k = None
    return k
    
urldata['tld'] = urldata['url'].apply(lambda i: get_tld_Name(i))

def tld_length(tld):
    try:
        return len(tld)
    except:
        return -1

urldata['tld_length'] = urldata['tld'].apply(lambda i: tld_length(i))

In [ ]:
#First Directory Length
def fd_length(url):
    try:
        urlpath= get_tld(url, as_object = True, fail_silently=False, fix_protocol=True).parsed_url.path
        return len(urlpath.split('/')[1])
    except:
        return 0

urldata['fd_length'] = urldata['url'].apply(lambda i: fd_length(i))

In [ ]:
# Query Length
def Query_Length(url):
    global total 
    try:
        Query = get_tld(url, as_object = True, fail_silently=False, fix_protocol=True).parsed_url.query
        return len(Query)
    except:
        return 0
#         total += 1
        

urldata['QueryLength'] = urldata['url'].apply(lambda i: Query_Length(i))

In [ ]:
# Gives number of '/' in URL (URL_Depth)

def getDepth(url):
    try:
        res = get_tld(url, as_object = True, fail_silently=False, fix_protocol=True)
        s = res.parsed_url.path.split('/')
        depth = 0
        for j in range(len(s)):
            if len(s[j]) != 0:
                depth = depth+1
    except: 
        depth = 0
    return depth

urldata['getDepth'] = urldata['url'].apply(lambda i: getDepth(i))

In [ ]:
def getQuery_Comp(url):
    try:
        res = get_tld(url, as_object = True, fail_silently=False, fix_protocol=True)
        s = res.parsed_url.query.split("&")
        return len(s)
    except: 
        return 0

urldata['getQCompon'] = urldata['url'].apply(lambda i: getQuery_Comp(i))
# res = get_tld('yourbittorrent.com/?q=anthony-hamilton-soulife', as_object = True, fail_silently=False, fix_protocol=True)
# res.parsed_url.query.split("&")

In [ ]:
urldata.head(10)

## Count Features


In [ ]:
feature = ['@','?','-','=','.','%','+','$','http', 'https', 'www']
for a in feature:
    urldata[a] = urldata['url'].apply(lambda i: i.count(a))

In [ ]:
def digit_count(url):
    digits = 0
    for i in url:
        if i.isnumeric():
            digits = digits + 1
    return digits
urldata['count_digits']= urldata['url'].apply(lambda i: digit_count(i))

def letter_count(url):
    letters = 0
    for i in url:
        if i.isalpha():
            letters = letters + 1
    return letters
urldata['count_letters']= urldata['url'].apply(lambda i: letter_count(i))

# def no_of_dir(url):  #the same as get_depth !!!!!
#     urldir = urlparse(url).path
#     return urldir.count('/')

# urldata['count_dir'] = urldata['url'].apply(lambda i: no_of_dir(i))

def no_of_embed(url):
    try:
        res = get_tld(url, as_object = True, fail_silently=False, fix_protocol=True)
        urldir = res.parsed_url.path
        return urldir.count('//')
    except: 
        return 0

urldata['count_embed_domian'] = urldata['url'].apply(lambda i: no_of_embed(i))

In [ ]:
urldata.head()

**3- Binary Features**

In [ ]:
def having_ip_address(url):
    match = re.search(
        '(([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.'
        '([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\/)|'  # IPv4
        '(([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.'
        '([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\/)|'  # IPv4 with port
        '((0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\/)' # IPv4 in hexadecimal
        '(?:[a-fA-F0-9]{1,4}:){7}[a-fA-F0-9]{1,4}|'
        '([0-9]+(?:\.[0-9]+){3}:[0-9]+)|'
        '((?:(?:\d|[01]?\d\d|2[0-4]\d|25[0-5])\.){3}(?:25[0-5]|2[0-4]\d|[01]?\d\d|\d)(?:\/\d{1,2})?)', url)  # Ipv6
    if match:
        return 1
    else:
        return 0
    
urldata['having_ip_address'] = urldata['url'].apply(lambda i: having_ip_address(i))

In [ ]:
#listing shortening services
shortening_services = r"bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|" \
                      r"yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|" \
                      r"short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|" \
                      r"doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|db\.tt|" \
                      r"qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|q\.gs|is\.gd|" \
                      r"po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|x\.co|" \
                      r"prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|" \
                      r"tr\.im|link\.zip\.net"


# 8. Checking for Shortening Services in URL (Tiny_URL)
def tinyURL(url):
    match=re.search(shortening_services,url)
    if match:
        return 1
    else:
        return 0
urldata['tinyURL'] = urldata['url'].apply(lambda i: tinyURL(i))

In [ ]:
def suspicious_words(url):
    match = re.search('PayPal|login|signin|bank|account|update|free|lucky|service|bonus|ebayisapi|webscr',
                      url)
    if match:
        return 1
    else:
        return 0
urldata['sus_url'] = urldata['url'].apply(lambda i: suspicious_words(i))

In [ ]:
# 9.Checking for Prefix or Suffix Separated by (-) in the Domain (Prefix/Suffix)
def prefixSuffix(url):
    try:
        res = get_tld(url, as_object = True, fail_silently=False,fix_protocol=True)
        if '-' in res.parsed_url.netloc:
            return 1            # phishing
        else:
            return 0            # legitimate
    except :
        #if there is no hostname ????? 
        return 1

    
urldata['prefixSuffix'] = urldata['url'].apply(lambda i: prefixSuffix(i))

In [ ]:
total = 0
def abnormal_url(url):
    try:
        hostname = get_tld(url, as_object = True, fail_silently=False,fix_protocol=True).parsed_url.netloc
        hostname = str(hostname)
        match = re.search(hostname, url)
        if match:
            # print match.group()
            return 1
        else:
            # print 'No matching pattern found'
            return 0
    except:
        return 0
#         total += 1
        

urldata['abnormal_url'] = urldata['url'].apply(lambda i: abnormal_url(i))
print(total)

#21054 not returned from get_tld

In [ ]:
def httpSecure(url):
    try:
        htp = get_tld(url, as_object = True, fail_silently=False,fix_protocol=True).parsed_url.scheme
        #It supports the following URL schemes: file , ftp , gopher , hdl , 
                               #http , https ... from urllib.parse
        match = str(htp)
        if match=='https':
            # print match.group()
            return 1
        else:
            # print 'No matching pattern found'
            return 0
    except:
        return 0
    
urldata['scheme_https'] = urldata['url'].apply(lambda i: httpSecure(i))

In [ ]:
urldata.head(10)

**4-Address Bar Based Features:**

## HTML and JavaScript based 

In [ ]:
# 14.End time of domain: The difference between termination time and current time (Domain_End) 
'''def domainEnd(domain_name):
  expiration_date = domain_name.expiration_date
  if isinstance(expiration_date,str):
    try:
      expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
    except:
      return 1
  if (expiration_date is None):
      return 1
  elif (type(expiration_date) is list):
      return 1
  else:
    today = datetime.now()
    end = abs((expiration_date - today).days)
    if ((end/30) < 6):
      end = 0
    else:
      end = 1
  return end
urldata['domainEnd'] = urldata['url'].apply(lambda i: domainEnd(i))
'''

In [ ]:
urldata.head()

## Build the model

In [ ]:
#Predictor Variables
X = urldata[['url_length','hostname_length', 'path_length', 'tld_length',  'fd_length', 'getDepth',
             '@', '?','-','=','.','%','+','$','http', 'https', 'count_digits', 'count_letters', 
             'having_ip_address', 'tinyURL', 'sus_url', 'QueryLength'
            ]].values 

#Target Variable
y = urldata['Class'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2,shuffle=True, random_state=5)

In [ ]:
print(len(X_train))

## Random forest model

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

#Random Forest
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)

rfc_predictions = rfc.predict(X_test)
print(accuracy_score(y_test, rfc_predictions)*100)

## Neural Network Model

In [ ]:
from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler

X = torch.Tensor(X)
y = torch.Tensor(y)
# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(X, y)

# Calculate the number of samples to include in each set.
train_size = int(0.6 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

In [ ]:
batch_size = 1024

# take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # get batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [ ]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.hid1 = torch.nn.Linear(28, 64)  # 4-(8-8)-1
        self.hid2 = torch.nn.Linear(64, 64)
        self.oupt = torch.nn.Linear(64, 1)
        self.relu = nn.ReLU()
        
        self.dropout = nn.Dropout(p=0.4)
        self.batchnorm1 = nn.BatchNorm1d(64)
        self.batchnorm2 = nn.BatchNorm1d(64)
        
        torch.nn.init.xavier_uniform_(self.hid1.weight)
        torch.nn.init.zeros_(self.hid1.bias)
        torch.nn.init.xavier_uniform_(self.hid2.weight)
        torch.nn.init.zeros_(self.hid2.bias)
        torch.nn.init.xavier_uniform_(self.oupt.weight)
        torch.nn.init.zeros_(self.oupt.bias)

    def forward(self, x):
        z = self.relu(self.hid1(x))
        z = self.batchnorm1(z)
        z = self.relu(self.hid2(z))
        z = self.batchnorm2(z)
        z = self.dropout(z)
#         z = self.oupt(z)
#         z = torch.tanh(self.hid2(z))
        z = torch.sigmoid(self.oupt(z))
        return z
  

device = torch.device("cpu")
torch.manual_seed(1)  # for initialization repro
net = Net().to(device)
print(net)

In [ ]:
lr= 0.001

criterion = nn.BCELoss()
# criterion =  nn.BCEWithLogitsLoss()

optimizer = torch.optim.Adam(net.parameters(), lr=lr)

# function to predict accuracy
def acc(pred,label):
    pred = torch.round(pred.squeeze())
    return torch.sum(pred == label.squeeze()).item()

# def binary_acc(y_pred, y_test):
#     y_pred_tag = torch.round(torch.sigmoid(y_pred))

#     correct_results_sum = (y_pred_tag == y_test).sum().float()
#     acc = correct_results_sum/y_test.shape[0]
#     acc = torch.round(acc * 100)
    
#     return acc

In [ ]:
def forward_back_prop(model, optimizer, criterion, batch):
    clip = 5
    # input_ids, attention_masks, token_type_ids = np.array(input_ids, dtype="int32"), np.array(attention_masks, dtype="int32"), np.array(token_type_ids, dtype="int32")
    # move data to GPU, if available
    input_features = batch[0].to(device)
    target = batch[1].to(device)

    optimizer.zero_grad()    
    output = model.forward(input_features)

    # calculate the loss and perform backprop
    loss = criterion(output.squeeze(), target.float())
    loss.backward()
    
    # calculating accuracy
    accuracy = acc(output,target)
    
    # clip_grad_norm prevent the exploding gradient problem in RNNs / LSTMs.
    nn.utils.clip_grad_norm_(model.parameters(), clip)
    optimizer.step()

    # return the loss over a batch and the hidden state produced by our model and accuracy
    return loss.item(), accuracy


In [ ]:
epochs = 50
valid_loss_min = np.Inf

# train for some number of epochs
epoch_tr_loss,epoch_vl_loss = [],[]
epoch_tr_acc,epoch_vl_acc = [],[]

for epoch in range(1, epochs+1):
    train_losses = []
    train_acc = 0.0
    net.train()
    # initialize hidden state
#     hidden = net.init_hidden(batch_size)

    for batch_i, batch in enumerate(train_dataloader, 1):

        # make sure iterate over completely full batches, only
        n_batches = len(train_dataloader.dataset)//batch_size
        if(batch_i > n_batches):
            break

        # forward, back prop
        loss, accuracy = forward_back_prop(net, optimizer, criterion, batch)  

        train_losses.append(loss)
        train_acc += accuracy
        
       
    #     val_h = model.init_hidden(batch_size)
    val_losses = []
    val_acc = 0.0
    net.eval()
    for batch_i, (input_ids, labels) in enumerate(validation_dataloader, 1):  
        # make sure iterate over completely full batches, only
        n_batches = len(validation_dataloader.dataset)//batch_size
        if(batch_i > n_batches):
            break


        input_ids, labels = input_ids.to(device), labels.to(device)
        output = net(input_ids)

        val_loss = criterion(output.squeeze(), labels.float())
        val_losses.append(val_loss.item())
        
        accuracy = acc(output,labels)
        val_acc += accuracy
            
    epoch_train_loss = np.mean(train_losses)
    epoch_val_loss = np.mean(val_losses)    
    epoch_train_acc = train_acc/len(train_dataloader.dataset)
    epoch_val_acc = val_acc/len(validation_dataloader.dataset)
    epoch_tr_loss.append(epoch_train_loss)
    epoch_vl_loss.append(epoch_val_loss)
    epoch_tr_acc.append(epoch_train_acc)
    epoch_vl_acc.append(epoch_val_acc)
    print(f'Epoch {epoch}') 
    print(f'train_loss : {epoch_train_loss} val_loss : {epoch_val_loss}')
    print(f'train_accuracy : {epoch_train_acc*100} val_accuracy : {epoch_val_acc*100}')
    if epoch_val_loss <= valid_loss_min:
#         torch.save(net.state_dict(), path)
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,epoch_val_loss))
        valid_loss_min = epoch_val_loss
    print(25*'==')

## LGBM Classifier

In [ ]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(boosting_type= 'gbdt',n_jobs = 10, 
          silent = True, random_state=5)
LGB_C = lgb.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn import metrics

y_pred = LGB_C.predict(X_test)
print(classification_report(y_test,y_pred))

score = metrics.accuracy_score(y_test, y_pred)
print("accuracy:   %0.3f" % score)

## Gradient Boosting Classifier

In [ ]:
# GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingClassifier
#clf = AdaBoostClassifier(n_estimators=100)
clf = GradientBoostingClassifier(n_estimators=200,learning_rate=1.0, max_depth=1, random_state=0).fit(X_train, y_train)
clf.score(X_test, y_test)

## Decision Tree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier

grid = DecisionTreeClassifier(random_state=42)
tree = grid.fit(X_train,y_train)
tree.score(X_test,y_test)

## Logistic Regression Model

In [ ]:
from sklearn.linear_model import LogisticRegression

#Logistic Regression
log_model = LogisticRegression()
log_model.fit(X_train,y_train)

log_predictions = log_model.predict(X_test)
accuracy_score(y_test,log_predictions)

## Naive Bayes Bernoulli Model

In [ ]:
from sklearn.naive_bayes import BernoulliNB

naive = BernoulliNB()

naive.fit(X_train,y_train)

naive_prediction = naive.predict(X_test)
accuracy_score(y_test,naive_prediction)

## Naive Bayes Gaussian Model

In [ ]:
from sklearn.naive_bayes import GaussianNB

naive = GaussianNB()

naive.fit(X_train,y_train)

naive_prediction = naive.predict(X_test)
accuracy_score(y_test,naive_prediction)